In [25]:
# ASABEL
# calculate the dGf for all molecules in grammar_2
# use smiles representation and turn into inchi keys, calculate energy

In [2]:
# import packages and dependencies
import csv
import functions_eQ 
from openbabel.pybel import readstring
from equilibrator_api import ComponentContribution, Q_, Reaction
from equilibrator_assets.generate_compound import create_compounds, get_or_create_compounds
from component_contribution.predict import GibbsEnergyPredictor

GP = GibbsEnergyPredictor()
cc = ComponentContribution()

In [ ]:
############# define input file name ##################
# define the csv to be opened from the data directory

# eductmolecules aka known molecules
# csv_molecules = "eductmols_grammar_2"

# unknown molecules created by MØD
csv_molecules = "look-up_unknown_gly_ala_2_5"

In [ ]:
############ intialize molecules dictionary ###################

# 
# Molecules Dictionary:
# -----------------
# molecules = {
#   name : smiles
# }
# 

import ast

# import csv file 
molecules = {}
with open(f'../data/{csv_molecules}.csv', mode='r') as infile:
    reader = csv.reader(infile)
    for rows in reader:
        name = rows[0]
        # when value is a list, choose smiles entry 
        values = ast.literal_eval(rows[1])
        molecules[name] = values[0]
    print(molecules)

In [ ]:
############ initialize get_standard_dgf_prime ############

### this is where the compound is created, queried, and calculated
# ## from eQ documentation
# for calculation of energy of formation
def get_standard_dgf_prime(cpd) -> Q_:
   reaction = Reaction({cpd: 1})
   return cc.standard_dg_prime(reaction)


### to create a look-up table with eQ inchi keys and energies create a new dictionary
# Look-up Dictionary:
# -------------------
# eQmolecules = {
#   name : [smiles, inchi, energy]
# }
#


# Update Dictionary:
# -------------------
# def getEQObjectOfMolecule(mol)
#   object_cpd = get_or_create_compounds(cc.ccache, [smiles], mol_format="smiles")
#   energy = get_standard_dgf_prime(object_cpd)
#   inchi_key = object.inchi_key()
#   eQmolecules[molName] = [smiles, inchi, energy]

In [18]:
############ get or create ############

def getEQObjectOfMolecule(molName, dict_name):
# finds or 
# requires molecules in dictonary as moleclues{ name : smiles}
    smiles = molecules[molName]
    object_cpd = get_or_create_compounds(cc.ccache, [smiles], mol_format="smiles")[0].compound
    print(object_cpd)
    energy = get_standard_dgf_prime(object_cpd)
    if object_cpd.id < 0:
        print("Compound not found in equilibrator-cache")
        print(f"∆fG'° of generated compound: {get_standard_dgf_prime(object_cpd)}")
    else:
        print(f"Found compound in equilibrator-cache: ID = {object_cpd.id}, name = {object_cpd.get_common_name()}")
        print(f"∆fG'° of cached compound: {get_standard_dgf_prime(object_cpd)}")
    inchi_key = object_cpd.inchi_key
    dict_name[molName] = [smiles, inchi_key, energy]


In [ ]:
############ get or create ############

## create look-up dictionary
eQmolecules_getorcreate = {}

for mol in molecules:
    getEQObjectOfMolecule(mol, eQmolecules_getorcreate)

print(eQmolecules_getorcreate)


In [39]:
### save Look-up Dictionary as external file

def create_look_up_energies(mols_dict, name):
    with open(f'../data/{name}.csv', 'w') as f:
        writer = csv.writer(f)
        for molName, data in mols_dict.items():
             writer.writerow([molName, data[0], data[1], data[2]])

create_look_up_energies(eQmolecules, f"{csv_molecules}_get_or_create")


In [ ]:
############ create ############

## version 17.12.24

## create look-up dictionary
eQmolecules_create = {}

## create lists from the molecules dict to iterate over the indices
# keys = original graphDFS of molecules created in MOD
mol_keys = [v for v in molecules.keys()]
# mols = smiles string with replaced CoA labels for input in eQuilibrator
mols = [molecules[k] for k in mol_keys]

## get length of mols list to iterate over indices
L = len(mols)

print(molecules.keys())
print(mol_keys)

## !! CREATE COMPOUNDS (eQuilibrator) from the mols list of smiles strings, creates a list of compound objects with different attributes
object_cpds = create_compounds(mols, mol_format="smiles", bypass_chemaxon=True, save_empty_compounds=True)

# check if all lists are the same length
print(len(object_cpds))
print(len(mols))
print(len(mol_keys))

## !! CALCULATE ENERGIES (eQuilibrator) using standard_dgf_prime for each compound object in created compounds
for i in range(0,L):
	en = get_standard_dgf_prime(object_cpds[i].compound)
	# create dictionary for look-up table
	# contains originial graphDFS as key: energy, smiles, inchi_key, status (valid OR bypass), method (chemaxon OR database)
	eQmolecules_create[mol_keys[i]] = [en,object_cpds[i].structure, object_cpds[i].inchi_key, object_cpds[i].status, object_cpds[i].method]
	print(object_cpds[i].structure, en)
	# check that indices of the lists are aligned
	if object_cpds[i].structure != mols[i]:
		print("ERROR")


print(len(eQmolecules_create.values()))

In [ ]:
print(eQmolecules_create)

In [ ]:
### create look-up table ###
# # save Look-up Dictionary as external file
#
def create_look_up_energies(mols_dict, name):
    with open(f'../data/{name}.csv', 'w') as f:
        writer = csv.writer(f)
        for molName, data in mols_dict.items():
            # look up tabel with original graphDFS, energy value, error value, smiles string, status, method
            writer.writerow([molName, data[0].value.m, data[0].error.m, data[1], data[2], data[3], data[4]])
            #  print(data[0].value.m,",",data[0].error.m)

create_look_up_energies(eQmolecules_create, f"{csv_molecules}_create_correct")


In [27]:
############ by Name ############

def getEQObjectOfMolecule_byName(molName, dict_name):
# finds OR creates a compound by NAME OF MOLECULE
# requires molecules in dictonary as moleclues{ name : smiles}   
    object_cpd = cc.search_compound(molName)
    print(object_cpd)
    energy = get_standard_dgf_prime(object_cpd)
    if object_cpd.id < 0:
        print("Compound not found in equilibrator-cache")
        print(f"∆fG'° of generated compound: {get_standard_dgf_prime(object_cpd)}")
    else:
        print(f"Found compound in equilibrator-cache: ID = {object_cpd.id}, name = {object_cpd.get_common_name()}")
        print(f"∆fG'° of cached compound: {get_standard_dgf_prime(object_cpd)}")
    inchi_key = object_cpd.inchi_key
    smiles = molecules[molName]
    dict_name[molName] = [smiles, inchi_key, energy]



In [ ]:
############ by Name ############

## create look-up dictionary
eQmolecules_byName = {}

for mol in molecules:
    getEQObjectOfMolecule_byName(mol, eQmolecules_byName)

print(eQmolecules_byName)


In [ ]:
# # save Look-up Dictionary as external file
#
def create_look_up_energies(mols_dict, name):
    with open(f'../data/{name}.csv', 'w') as f:
        writer = csv.writer(f)
        for molName, data in mols_dict.items():
             writer.writerow([molName, data[0], data[1], data[2]])

create_look_up_energies(eQmolecules_byName, f"{csv_molecules}_byName")


In [ ]:
## compare energies calculated or found in the database


### for when energy of compound is calculated to get the energy ouf of dictionary
# 
# Energy Getter Function:
# ----------------
# def getEnergy(molName):
#   return EQmolecules.get(molName)[2]

def getEnergy(molName, dict_name):
    return dict_name.get(molName)[2]

for mol in eQmolecules_getorcreate:
    print("-------------------------")
    print(mol)
    print(getEnergy(mol, eQmolecules_getorcreate)) 
    print(getEnergy(mol, eQmolecules_create)) 
    print(getEnergy(mol, eQmolecules_byName))


In [ ]:
# ---------- helpermolecules ----------

## import helpermols
help_molecules = {}
with open('../data/helpermols_grammar_2.csv', mode='r') as infile:
    reader = csv.reader(infile)
    for rows in reader:
        # print(rows[0])
        k = rows[0]
        v = rows[1]
        help_molecules[k] = v
    print(help_molecules)

############ by Name ############
def getEQObjectOfMolecule_byName(molName, dict_name, input_name):
# finds OR creates a compound by NAME OF MOLECULE
# requires molecules in dictonary as moleclues{ name : smiles}   
    object_cpd = cc.search_compound(molName)
    print(object_cpd)
    energy = get_standard_dgf_prime(object_cpd)
    if object_cpd.id < 0:
        print("Compound not found in equilibrator-cache")
        print(f"∆fG'° of generated compound: {get_standard_dgf_prime(object_cpd)}")
    else:
        print(f"Found compound in equilibrator-cache: ID = {object_cpd.id}, name = {object_cpd.get_common_name()}")
        print(f"∆fG'° of cached compound: {get_standard_dgf_prime(object_cpd)}")
    inchi_key = object_cpd.inchi_key
    smiles = input_name[molName]
    dict_name[molName] = [smiles, inchi_key, energy]

# look up by name
eQmolecules_help_byName = {}

for mol in help_molecules:
    getEQObjectOfMolecule_byName(mol, eQmolecules_help_byName, help_molecules)

print(eQmolecules_help_byName)


In [ ]:

# save look-up table
create_look_up_energies(eQmolecules_help_byName, "eQmolecules_helpermols_grammar_2_byName")

for mol in eQmolecules_help_byName:
    print("-------------------------")
    print(mol)
    print(getEnergy(mol, eQmolecules_help_byName))

